In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import glob

from sklearn.metrics import confusion_matrix
import itertools

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPool1D
from keras.initializers import random_uniform
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as SS

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Dense,LSTM,Dropout,Flatten,BatchNormalization,Conv1D,GlobalMaxPooling1D,MaxPooling1D
from keras.optimizers import  SGD
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing import sequence
from keras.layers import SimpleRNN
#from hyperas.distributions import uniform

from keras.utils.np_utils import to_categorical
from keras import regularizers
import pandas as pd
import string
import numpy as np
import matplotlib.pyplot as plt

# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')

# Data import

In [2]:
dataset = pd.read_csv(r'..')  # use your path

In [3]:
dataset.head()

DestinationPort  FlowDuration  TotalFwdPackets  TotalBackwardPackets  \
0             80.0     1293792.0              3.0                   7.0   
1             80.0     4421382.0              4.0                   0.0   
2             80.0     1083538.0              3.0                   6.0   
3             80.0    80034360.0              8.0                   4.0   
4             80.0      642654.0              3.0                   6.0   

   TotalLengthofFwdPackets  TotalLengthofBwdPackets  FwdPacketLengthMax  \
0                     26.0                  11607.0                20.0   
1                     24.0                      0.0                 6.0   
2                     26.0                  11601.0                20.0   
3                     56.0                  11601.0                20.0   
4                     26.0                  11607.0                20.0   

   FwdPacketLengthMin  FwdPacketLengthMean  FwdPacketLengthStd  ...  \
0                 0.0             8.666667           10.263203  ...   
1                 6.0             6.000000            0.000000  ...   
2                 0.0             8.666667           10.263203  ...   
3                 0.0             7.000000            5.656854  ...   
4                 0.0             8.666667           10.263203  ...   

   min_seg_size_forward  ActiveMean  ActiveStd  ActiveMax  ActiveMin  \
0                  20.0         0.0        0.0        0.0        0.0   
1                  20.0         0.0        0.0        0.0        0.0   
2                  20.0         0.0        0.0        0.0        0.0   
3                  20.0       939.0        0.0      939.0      939.0   
4                  20.0         0.0        0.0        0.0        0.0   

     IdleMean     IdleStd     IdleMax    IdleMin  Label  
0         0.0         0.0         0.0        0.0   DDoS  
1         0.0         0.0         0.0        0.0   DDoS  
2         0.0         0.0         0.0        0.0   DDoS  
3  39300000.0  44200000.0  70600000.0  8072664.0   DDoS  
4         0.0         0.0         0.0        0.0   DDoS  

[5 rows x 79 columns]

In [4]:
dataset.shape

(780444, 79)

In [5]:
# Splitting dataset into features and labels.
labels = dataset['Label']
features = dataset.loc[:, dataset.columns != 'Label'].astype('float64')

In [6]:
# For scaling the data, we use RobustScaler class from sklearn.

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

In [7]:
scaler = RobustScaler()
scaler.fit(features)

features = scaler.transform(features)

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
LE = LabelEncoder()

LE.fit(labels)
labels = LE.transform(labels)

In [10]:
# Labels have been replaced with integers.

np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [11]:
# Checking that encoding reversal works.

d = LE.inverse_transform(labels)
d = pd.Series(d)
d.unique()


array(['DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web_Attack_Brute_Force', 'Web_Attack_XSS',
       'Web_Attack_Sql_Injection', 'FTPPatator', 'SSHPatator',
       'DoS_slowloris', 'DoS_Slowhttptest', 'DoS_Hulk', 'DoS_GoldenEye',
       'Heartbleed', 'BENIGN'], dtype=object)

### Splitting the data

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# The next step is to split training and testing data. For this we will use sklearn function train_test_split().

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size= 0.3,random_state=1)

In [14]:
features_train.shape, labels_train.shape ,features_test.shape , labels_test.shape

((546310, 78), (546310,), (234134, 78), (234134,))

In [15]:
# this function has evaluation metrics
def report(prediction, y_true):
    print('        ----------Classification Report Of Classes-------------')
    print(classification_report(y_true,prediction))
    print('\n        ----------Validation Data------------------')
    print('Accuarcy:',accuracy_score(y_true, prediction)* 100)
    print('Precision: {:,.4f} %'.format(precision_score(y_true, prediction, average='weighted') * 100))
    print('Recall-score: {:,.4f}'.format(recall_score(y_true, prediction, average='weighted') * 100))
    print('F1-score: {:,.4f}'.format(f1_score(y_true, prediction, average='weighted') * 100))

In [17]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

### Logistic Regression

In [18]:
ln = LogisticRegression()

In [19]:
ln.fit(features_train,labels_train)

LogisticRegression()

In [20]:
predict = ln.predict(features_test)
report(predict,labels_test)

        ----------Classification Report Of Classes-------------
              precision    recall  f1-score   support

           0       0.72      0.75      0.74    117310
           1       0.00      0.00      0.00       594
           2       0.66      0.33      0.44     38037
           3       0.03      0.32      0.05      3072
           4       0.71      0.25      0.37     51585
           5       0.00      0.00      0.00      1557
           6       0.00      0.07      0.01      1618
           7       0.00      0.00      0.00      1635
           8       0.00      0.00      0.00         2
           9       0.00      0.00      0.00        13
          10       0.00      0.00      0.00     17214
          11       0.00      0.00      0.00       883
          12       0.00      0.00      0.00       408
          13       0.00      0.00      0.00         5
          14       0.00      0.00      0.00       201

    accuracy                           0.49    234134
   macro avg    

### GaussianNB

In [21]:
nb = GaussianNB()

In [22]:
nb.fit(features_train,labels_train)

GaussianNB()

In [23]:
predict = nb.predict(features_test)
report(predict,labels_test)

        ----------Classification Report Of Classes-------------
              precision    recall  f1-score   support

           0       1.00      0.00      0.00    117310
           1       0.07      0.31      0.12       594
           2       0.40      0.14      0.21     38037
           3       0.12      0.28      0.17      3072
           4       0.00      0.00      0.00     51585
           5       0.25      0.55      0.34      1557
           6       0.24      0.52      0.33      1618
           7       0.01      1.00      0.02      1635
           8       0.33      0.50      0.40         2
           9       0.01      0.38      0.02        13
          10       0.14      0.01      0.03     17214
          11       0.00      0.00      0.00       883
          12       0.00      0.00      0.00       408
          13       0.00      0.00      0.00         5
          14       0.00      0.00      0.00       201

    accuracy                           0.04    234134
   macro avg    

### AdaBoostClassifier

In [24]:
ab = AdaBoostClassifier(n_estimators=100)

In [25]:
ab.fit(features_train,labels_train)

AdaBoostClassifier(n_estimators=100)

In [26]:
predict = ab.predict(features_test)
report(predict,labels_test)

        ----------Classification Report Of Classes-------------
              precision    recall  f1-score   support

           0       0.74      0.99      0.85    117310
           1       0.00      0.00      0.00       594
           2       0.00      0.00      0.00     38037
           3       0.00      0.00      0.00      3072
           4       0.64      0.90      0.75     51585
           5       0.00      0.00      0.00      1557
           6       0.00      0.00      0.00      1618
           7       0.47      1.00      0.64      1635
           8       0.00      0.00      0.00         2
           9       0.00      0.00      0.00        13
          10       0.00      0.00      0.00     17214
          11       0.00      0.00      0.00       883
          12       0.00      0.00      0.00       408
          13       0.00      0.00      0.00         5
          14       0.00      0.00      0.00       201

    accuracy                           0.70    234134
   macro avg    

### Decision Tree Classifier

In [27]:
dt = DecisionTreeClassifier()

In [28]:
dt.fit(features_train,labels_train)

DecisionTreeClassifier()

In [29]:
predict = dt.predict(features_test)
report(predict,labels_test)

        ----------Classification Report Of Classes-------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    117310
           1       0.95      0.91      0.93       594
           2       1.00      1.00      1.00     38037
           3       1.00      0.99      0.99      3072
           4       1.00      1.00      1.00     51585
           5       0.99      0.98      0.99      1557
           6       0.98      1.00      0.99      1618
           7       1.00      1.00      1.00      1635
           8       0.67      1.00      0.80         2
           9       0.89      0.62      0.73        13
          10       1.00      1.00      1.00     17214
          11       1.00      1.00      1.00       883
          12       0.69      0.71      0.70       408
          13       0.29      0.40      0.33         5
          14       0.39      0.38      0.38       201

    accuracy                           1.00    234134
   macro avg    

### RandomForestClassifier

In [30]:
rf = RandomForestClassifier(n_estimators=100)

In [31]:
rf.fit(features_train,labels_train)

RandomForestClassifier()

In [32]:
predict = rf.predict(features_test)
report(predict,labels_test)

        ----------Classification Report Of Classes-------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    117310
           1       0.95      0.91      0.93       594
           2       1.00      1.00      1.00     38037
           3       1.00      1.00      1.00      3072
           4       1.00      1.00      1.00     51585
           5       1.00      0.99      0.99      1557
           6       0.99      1.00      0.99      1618
           7       1.00      1.00      1.00      1635
           8       1.00      0.50      0.67         2
           9       1.00      0.62      0.76        13
          10       1.00      1.00      1.00     17214
          11       1.00      1.00      1.00       883
          12       0.71      0.81      0.75       408
          13       1.00      0.20      0.33         5
          14       0.46      0.32      0.38       201

    accuracy                           1.00    234134
   macro avg    